In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from backtest import backtest
from plotting import plot_fills
from downloader import Downloader
from pure_funcs import (
    denumpyize,
    numpyize,
    candidate_to_live_config,
    calc_spans,
    analyze_fills,
    create_xk,
    round_dynamic,
    round_values,
)
from njit_funcs import *
from procedures import (
    dump_live_config,
    load_live_config,
    add_argparse_args,
    prepare_backtest_config,
    make_get_filepath,
)
from time import time
import sys
import argparse
import pprint
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np

In [4]:
plt.rcParams["figure.figsize"] = [29, 18]
plt.rcParams['figure.facecolor'] = 'w'
pd.set_option("precision", 10)

In [5]:
class Args:
    def __init__(self):
        self.backtest_config_path = "configs/backtest/default.hjson"
        self.exchange = "binance"
        self.symbol = "XMRUSDT"
        self.market_type = "futures"
        self.user = "binance_01"
        self.start_date = "2021-08-01"
        self.end_date = "2021-08-29"
        self.starting_balance = 10000.0
        self.starting_configs = ""
        self.base_dir = "backtests"


config = await prepare_backtest_config(Args())
dl = Downloader(config)
sts = time()
data = await dl.get_sampled_ticks()
timestamps = data[:, 0]
qtys = data[:, 1]
prices = data[:, 2]
config["n_days"] = (timestamps[-1] - timestamps[0]) / (1000 * 60 * 60 * 24)

print(f"millis to load {len(prices)} ticks {(time() - sts) * 1000:.0f}ms")

fetching market_specific_settings...
linear perpetual
linear perpetual
2022-01-14T10:14:35  fetched ticks XMRUSDT 1 2020-02-03T15:03:06.871000 
2022-01-14T10:14:35  Fetching XMRUSDT 2021-08 
2022-01-14T10:14:45  Saving file 40123032_40199999_1627776002386_1627845965056.csv 2021-08-01T07:00:02.386000 
2022-01-14T10:14:45  Saving file 40200000_40299999_1627845966666_1627935044183.csv 2021-08-02T02:26:06.666000 
2022-01-14T10:14:46  Saving file 40300000_40399999_1627935047594_1628035431332.csv 2021-08-03T03:10:47.594000 
2022-01-14T10:14:46  Saving file 40400000_40499999_1628035431332_1628135106521.csv 2021-08-04T07:03:51.332000 
2022-01-14T10:14:47  Saving file 40500000_40599999_1628135106521_1628210828907.csv 2021-08-05T10:45:06.521000 
2022-01-14T10:14:47  Saving file 40600000_40699999_1628210828907_1628286437106.csv 2021-08-06T07:47:08.907000 
2022-01-14T10:14:48  Saving file 40700000_40799999_1628286437106_1628362886354.csv 2021-08-07T04:47:17.106000 
2022-01-14T10:14:48  Saving file

CancelledError: 

In [ ]:
df = pd.DataFrame({"timestamp": timestamps, "qty": qtys, "price": prices}).set_index(
    "timestamp"
)
df.price.iloc[::100].plot(title="Coin Price", xlabel="Time", ylabel="Price")

In [ ]:
# choose a slice on which to test
wsize_days = 240
ts = int(data[-1][0] - 60 * 60 * 24 * 1000 * wsize_days)
idx = np.argmax(data[:, 0] >= ts)
dataslice = data[idx:]

In [ ]:
hand_tuned = {
    "config_name": "hand_tuned",
    "logging_level": 0,
    "long": {
        "enabled": True,
        "grid_span": 0.17,
        "pbr_limit": 1.7,
        "max_n_entry_orders": 10,
        "initial_qty_pct": 0.01,
        "eprice_pprice_diff": 0.0025,
        "secondary_pbr_allocation": 0.5,
        "secondary_pprice_diff": 0.35,
        "eprice_exp_base": 1.618034,
        "min_markup": 0.0045,
        "markup_range": 0.0075,
        "n_close_orders": 7,
    },
    "shrt": {
        "enabled": True,
        "grid_span": 0.17,
        "pbr_limit": 1.7,
        "max_n_entry_orders": 10,
        "initial_qty_pct": 0.01,
        "eprice_pprice_diff": 0.0025,
        "secondary_pbr_allocation": 0.5,
        "secondary_pprice_diff": 0.35,
        "eprice_exp_base": 1.618034,
        "min_markup": 0.0045,
        "markup_range": 0.0075,
        "n_close_orders": 7,
    },
}

# round for better human readability
hand_tuned = round_values(hand_tuned, 4)
config["starting_balance"] = Args().starting_balance
config["latency_simulation_ms"] = 1000
config_to_test = {**config, **numpyize(hand_tuned)}
dump_live_config(config_to_test, make_get_filepath(f"tmp/hand_tuned.json"))
hand_tuned

In [ ]:
sts = time()
fills, stats = backtest(config_to_test, dataslice)
elapsed = time() - sts
print(f"seconds elapsed {elapsed:.4f}")
fdf, sdf, analysis = analyze_fills(fills, stats, config_to_test)
pprint.pprint(analysis)
fdf

In [ ]:
sdf

In [ ]:
sdf.balance.plot()
sdf.equity.plot(title="Balance and equity", xlabel="Time", ylabel="Balance")

In [ ]:
plot_fills(df, fdf, plot_whole_df=True, title="Overview Fills")

In [ ]:
fdf[fdf.psize > 0.0].psize.plot(
    title="Position size in terms of contracts",
    xlabel="Time",
    ylabel="Long Position size",
)
fdf[fdf.psize < 0.0].psize.plot(
    title="Position size in terms of contracts",
    xlabel="Time",
    ylabel="Short Position size",
)

In [ ]:
sdf.price.plot(title="Average entry price", xlabel="Time", ylabel="Price")
sdf.long_pprice.replace(0.0, np.nan).plot()
sdf.shrt_pprice.replace(0.0, np.nan).plot()

In [ ]:
lpprices = sdf[sdf.long_pprice != 0.0]
pa_distance_long = (lpprices.long_pprice - lpprices.price).abs() / lpprices.price
print(f"Mean price action distance long {pa_distance_long.mean():.6f}")
pa_distance_long.plot(
    title="Price action distance", xlabel="Time", ylabel="Price action distance"
)

In [ ]:
spprices = sdf[sdf.shrt_pprice != 0.0]
pa_distance_shrt = (spprices.shrt_pprice - spprices.price).abs() / spprices.price
print(f"Mean price action distance short {pa_distance_shrt.mean():.6f}")
pa_distance_shrt.plot(
    title="Price action distance", xlabel="Time", ylabel="Price action distance"
)

In [ ]:
balance = config_to_test["starting_balance"]
highest_bid = df.price.iloc[-1]

inverse = config_to_test["inverse"]
do_long = True
qty_step = config_to_test["qty_step"]
price_step = config_to_test["price_step"]
min_qty = config_to_test["min_qty"]
min_cost = config_to_test["min_cost"]
c_mult = config_to_test["c_mult"]

grid_span = config_to_test["long"]["grid_span"]
pbr_limit = config_to_test["long"]["pbr_limit"]
max_n_entry_orders = config_to_test["long"]["max_n_entry_orders"]
initial_qty_pct = config_to_test["long"]["initial_qty_pct"]
eprice_pprice_diff = config_to_test["long"]["eprice_pprice_diff"]
eprice_exp_base = config_to_test["long"]["eprice_exp_base"]
secondary_pbr_allocation = config_to_test["long"]["secondary_pbr_allocation"]
secondary_pprice_diff = config_to_test["long"]["secondary_pprice_diff"]

grid = calc_whole_long_entry_grid(
    balance,
    highest_bid,
    inverse,
    qty_step,
    price_step,
    min_qty,
    min_cost,
    c_mult,
    grid_span,
    pbr_limit,
    max_n_entry_orders,
    initial_qty_pct,
    eprice_pprice_diff,
    secondary_pbr_allocation,
    secondary_pprice_diff,
    eprice_exp_base=eprice_exp_base,
)
gdf = pd.DataFrame(grid, columns=["qty", "price", "psize", "pprice", "pbr"])

gdf.loc[:, "eprice_pprice_diff"] = abs(gdf.price - gdf.pprice) / gdf.price
gdf.loc[:, "ddown_factor"] = gdf.qty / gdf.psize.shift()
gdf.loc[:, "bkr_price"] = gdf.apply(
    lambda x: calc_bankruptcy_price(
        balance, x.psize, x.pprice, 0.0, 0.0, inverse, c_mult
    ),
    axis=1,
)
colors = "rbygcmk"

# Display grid on graph
# -- = pprice = new position price
# - = price = DCA entry
timedt = pd.to_datetime(timestamps, unit="ms")
df = pd.DataFrame({"timestamp": timedt, "qty": qtys, "price": prices}).set_index(
    "timestamp"
)
lastdayfrom = pd.to_datetime(Args().end_date)
# lastdayfrom = pd.Timestamp('2021-05-21')
plt = (
    df.price.loc[lastdayfrom - pd.Timedelta(days=280) : lastdayfrom]
    .iloc[::100]
    .plot(title="Grid position", xlabel="Time", ylabel="Price")
)
for i, e in enumerate(gdf.itertuples()):
    plt.axhline(y=e.price, color=f"{colors[i%len(colors)]}", linestyle="-")
    #plt.axhline(y=e.pprice, color=f"{colors[i%len(colors)]}", linestyle="--")


In [ ]:
a = (1 - gdf.iloc[[0, -1]]["price"].iat[-1] / gdf.iloc[[0, -1]]["price"].iat[0]) * 100
print("\nLong Grid Span = {:.2f} %\n".format(round(a, 2)))
gdf

In [ ]:
balance = config_to_test["starting_balance"]
lowest_ask = df.price.iloc[-1]

inverse = config_to_test["inverse"]
do_shrt = True
qty_step = config_to_test["qty_step"]
price_step = config_to_test["price_step"]
min_qty = config_to_test["min_qty"]
min_cost = config_to_test["min_cost"]
c_mult = config_to_test["c_mult"]

grid_span = config_to_test["shrt"]["grid_span"]
pbr_limit = config_to_test["shrt"]["pbr_limit"]
max_n_entry_orders = config_to_test["shrt"]["max_n_entry_orders"]
initial_qty_pct = config_to_test["shrt"]["initial_qty_pct"]
eprice_pprice_diff = config_to_test["shrt"]["eprice_pprice_diff"]
eprice_exp_base = config_to_test["shrt"]["eprice_exp_base"]
secondary_pbr_allocation = config_to_test["shrt"]["secondary_pbr_allocation"]
secondary_pprice_diff = config_to_test["shrt"]["secondary_pprice_diff"]

grid = calc_whole_shrt_entry_grid(
    balance,
    lowest_ask,
    inverse,
    qty_step,
    price_step,
    min_qty,
    min_cost,
    c_mult,
    grid_span,
    pbr_limit,
    max_n_entry_orders,
    initial_qty_pct,
    eprice_pprice_diff,
    secondary_pbr_allocation,
    secondary_pprice_diff,
    eprice_exp_base=eprice_exp_base,
)
gdf = pd.DataFrame(grid, columns=["qty", "price", "psize", "pprice", "pbr"])

gdf.loc[:, "eprice_pprice_diff"] = abs(gdf.price - gdf.pprice) / gdf.price
gdf.loc[:, "ddown_factor"] = gdf.qty / gdf.psize.shift()
gdf.loc[:, "bkr_price"] = gdf.apply(
    lambda x: calc_bankruptcy_price(
        balance, x.psize, x.pprice, 0.0, 0.0, inverse, c_mult
    ),
    axis=1,
)
colors = "rbygcmk"

# Display grid on graph
# -- = pprice = new position price
# - = price = DCA entry
timedt = pd.to_datetime(timestamps, unit="ms")
df = pd.DataFrame({"timestamp": timedt, "qty": qtys, "price": prices}).set_index(
    "timestamp"
)
lastdayfrom = pd.to_datetime(Args().end_date)
# lastdayfrom = pd.Timestamp('2021-05-21')
plt = (
    df.price.loc[lastdayfrom - pd.Timedelta(days=280) : lastdayfrom]
    .iloc[::100]
    .plot(title="Grid position", xlabel="Time", ylabel="Price")
)
for i, e in enumerate(gdf.itertuples()):
    plt.axhline(y=e.price, color=f"{colors[i%len(colors)]}", linestyle="-")
    #plt.axhline(y=e.pprice, color=f"{colors[i%len(colors)]}", linestyle="--")


In [ ]:
a = (1 - gdf.iloc[[0, -1]]["price"].iat[-1] / gdf.iloc[[0, -1]]["price"].iat[0]) * 100
print("\nShort Grid Span = {:.2f} %\n".format(round(a, 2)))
gdf